In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [6]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [7]:
df['loc']

1     (8955 Collins Ave, Surfside, FL 33154, USA, (2...
2     (5272 Fisher Island Dr, Miami Beach, FL 33109,...
3     (1 Collins Ave, Miami Beach, FL 33139, USA, (2...
4     (10201 Collins Ave, Bal Harbour, FL 33154, USA...
5     (Santa Maria, 1643 Brickell Ave, Miami, FL 331...
6     (10201 Collins Ave, Bal Harbour, FL 33154, USA...
7     (1425 Brickell Ave #60am, Miami, FL 33131, USA...
8     (18975 Collins Ave, Sunny Isles Beach, FL 3316...
9     (765 Crandon Blvd # 100, Key Biscayne, FL 3314...
10    (520 West Ave, Miami Beach, FL 33139, USA, (25...
Name: loc, dtype: object

In [8]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [9]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Buyer Agent,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Arte Surfside\r8955 Collins Ave 802\rSurfside,3/12/2021,1/27/2022,286,"$11,200,000.00",3876,"$3,582.85",Miltiadis Kastanis,Ryan Mendell,Arte Surfside 8955 Collins Ave 802 Surfside,Arte Surfside,8955 Collins Ave 802 Surfside,"(8955 Collins Ave, Surfside, FL 33154, USA, (2...","(25.8758096, -80.12120449999999, 0.0)",25.875810,-80.121204,0.0
2,Bayview\r5272 Fisher Island Dr 5272\rMiami Beach,2/28/2020,1/24/2022,479,"$8,100,000.00",6700,"$1,208.96",Elena Bluntzer,Elena Bluntzer,Bayview 5272 Fisher Island Dr 5272 Miami Beach,Bayview,5272 Fisher Island Dr 5272 Miami Beach,"(5272 Fisher Island Dr, Miami Beach, FL 33109,...","(25.7626891, -80.1484152, 0.0)",25.762689,-80.148415,0.0
3,One Ocean\r1 Collins Ave 401\rMiami Beach,9/3/2021,1/27/2022,145,"$6,750,000.00",4390,"$1,910.02",Stacy Robins,Stacy Robins,One Ocean 1 Collins Ave 401 Miami Beach,One Ocean,1 Collins Ave 401 Miami Beach,"(1 Collins Ave, Miami Beach, FL 33139, USA, (2...","(25.7688413, -80.13400209999999, 0.0)",25.768841,-80.134002,0.0
4,Oceana Bal Harbour Condo\r10201 Collins Ave 14...,7/22/2021,1/24/2022,169,"$6,500,000.00",3345,"$1,943.20",Liz Hogan,Matias Derito,Oceana Bal Harbour Condo 10201 Collins Ave 140...,Oceana Bal Harbour Condo,10201 Collins Ave 1403 Bal Harbour,"(10201 Collins Ave, Bal Harbour, FL 33154, USA...","(25.8951514, -80.1230855, 0.0)",25.895151,-80.123086,0.0
5,Santa Maria Condo\r1643 Brickell Ave 3904\rMiami,1/27/2022,1/28/2022,1,"$4,200,000.00",4030,"$1,042.18",Ysabel Sanabria,Ysabel Sanabria,Santa Maria Condo 1643 Brickell Ave 3904 Miami,Santa Maria Condo,1643 Brickell Ave 3904 Miami,"(Santa Maria, 1643 Brickell Ave, Miami, FL 331...","(25.7554109, -80.1950473, 0.0)",25.755411,-80.195047,0.0
6,Ritz Carlton Harbour\r10203 Collins Ave 1705\r...,5/5/2021,1/26/2022,226,"$3,345,000.00",2221,"$1,506.08",Rachel Benmeir,Billy Hernandez,Ritz Carlton Harbour 10203 Collins Ave 1705 Ba...,Ritz Carlton Harbour,10203 Collins Ave 1705 Bal Harbour,"(10201 Collins Ave, Bal Harbour, FL 33154, USA...","(25.8951514, -80.1230855, 0.0)",25.895151,-80.123086,0.0
7,Four Seasons Residences\r1425 Brickell Ave 60A...,11/10/2021,1/24/2022,8,"$3,200,000.00",3554,$900.39,Fabian Dominguez,Sylvia Chamorro,Four Seasons Residences 1425 Brickell Ave 60AM...,Four Seasons Residences,1425 Brickell Ave 60AM Miami,"(1425 Brickell Ave #60am, Miami, FL 33131, USA...","(25.7591323, -80.1918295, 0.0)",25.759132,-80.191829,0.0
8,Armani Casa Residences\r18975 Collins Ave 2504...,12/17/2021,1/26/2022,20,"$2,550,000.00",1608,"$1,585.82",Barbara Savalli,Sylvia Fragos,Armani Casa Residences 18975 Collins Ave 2504 ...,Armani Casa Residences,18975 Collins Ave 2504 Sunny Isles Beach,"(18975 Collins Ave, Sunny Isles Beach, FL 3316...","(25.9519943, -80.11997, 0.0)",25.951994,-80.119970,0.0
9,Club Tower Two Condo\r785 Crandon Blvd 404\rKe...,5/26/2021,1/28/2022,203,"$2,190,000.00",2500,$876.00,Christian Balbontin,Alessandra Delpasso,Club Tower Two Condo 785 Crandon Blvd 404 Key ...,Club Tower Two Condo,785 Crandon Blvd 404 Key Biscayne,"(765 Crandon Blvd # 100, Key Biscayne, FL 3314...","(25.6868928, -80.16027969999999, 0.0)",25.686893,-80.160280,0.0
10,The Bentley Bay Condo\r520 West Ave 2102\rMiam...,11/1/2021,1/27/2022,46,"$1,750,000.00",1929,$907.21,Eduardo Bleiberg,David Vazquez,The Bentley Bay Condo 520 West Ave 2102 Miami ...,The Bentley Bay Condo,520 West Ave 2102 Miami Beach,"(520 West Ave, Miami Beach, FL 33139, USA, (25...","(25.7752309, -80.1419425, 0.0)",25.775231,-80.141942,0.0


Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [10]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Buyer Agent',
       'geo_address', 'building_name', 'address_only', 'loc', 'point', 'lat',
       'lon', 'altitude'],
      dtype='object')

In [11]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('January 23rd - January 29th')

marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [12]:
m.save('index.html')